<a href="https://colab.research.google.com/github/William-Schwarz/Decifra-Contrato/blob/william_ds/Decifra_Contrato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:

!pip -q install google-genai google-adk PyPDF2 python-docx

In [146]:
import os
from google.api_core import exceptions
from google.colab import userdata
from google import genai
from google.colab import files
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings
import PyPDF2

warnings.filterwarnings("ignore")

In [147]:
# Configura a API Key do Google Gemini
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except Exception as e:
    print(f"⚠️ Erro ao configurar a chave da API do Google Gemini: {e}")
    exit()

In [148]:
# Configura o cliente da SDK do Gemini
try:
    client = genai.Client()
    MODEL_ID = "gemini-2.0-flash"
except Exception as e:
    print(f"⚠️ Erro ao inicializar o cliente Gemini: {e}")
    exit()

In [149]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    try:
        for event in runner.run(user_id="user1", session_id="session1", new_message=content):
            if event.is_final_response():
                for part in event.content.parts:
                    if part.text is not None:
                        final_response += part.text
                        final_response += "\n"
    except exceptions.ServiceUnavailable as e:
        return f"⚠️ Serviço indisponível ao comunicar com o agente '{agent.name}'. Por favor, tente novamente mais tarde. Detalhes: {e}"
    except exceptions.InvalidArgument as e:
        return f"⚠️ Argumento inválido fornecido ao agente '{agent.name}'. Verifique a entrada. Detalhes: {e}"
    except Exception as e:
        return f"⚠️ Erro ao comunicar com o agente '{agent.name}': {e}"
    return final_response

In [150]:
# Função auxiliar que faz a leitura do arquivo
def ler_arquivo(nome_arquivo):
    try:
        if nome_arquivo.endswith(".pdf"):
            with open(nome_arquivo, 'rb') as file:
                try:
                    reader = PyPDF2.PdfReader(file)
                    text = ""
                    for page_num in range(len(reader.pages)):
                        page = reader.pages[page_num]
                        text += page.extract_text()
                    return text
                except Exception as e:
                    return f"⚠️ Erro ao ler o conteúdo do arquivo PDF '{nome_arquivo}': {e}"
        elif nome_arquivo.endswith(".docx"):
            try:
                from docx import Document
                document = Document(nome_arquivo)
                text = ""
                for paragraph in document.paragraphs:
                    text += paragraph.text + "\n"
                return text
            except ImportError:
                return "⚠️ A biblioteca 'python-docx' não está instalada. Por favor, instale-a com: pip install python-docx"
            except Exception as e:
                return f"⚠️ Erro ao ler o conteúdo do arquivo DOCX '{nome_arquivo}': {e}"
        elif nome_arquivo.endswith(".txt"):
            with open(nome_arquivo, 'r', encoding='utf-8') as file:
                return file.read()
        else:
            return "⚠️ Formato de arquivo não suportado. Por favor, envie um arquivo .pdf, .docx ou .txt."
    except FileNotFoundError:
        return f"⚠️ Erro: Arquivo '{nome_arquivo}' não encontrado."
    except Exception as e:
        return f"⚠️ Erro ao processar o arquivo '{nome_arquivo}': {e}"

In [151]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [152]:
# Agente 1: Receptor e Extrator de Contrato
def agente_receptor():
    receptor = Agent(
        name="agente_receptor",
        model=MODEL_ID,
        instruction="""
        Você é o agente inicial do sistema Decifra-Contrato. Sua tarefa é receber o contrato fornecido pelo usuário.
        Este contrato pode ser um texto diretamente inserido ou o conteúdo de um arquivo.

        Uma vez que o contrato é fornecido, sua principal função é extrair o texto completo e limpo do contrato.
        Remova qualquer formatação desnecessária que possa dificultar a análise posterior (como quebras de linha excessivas, numerações automáticas, etc.).

        Se o usuário fornecer informações adicionais sobre o tipo de contrato (por exemplo, "contrato de aluguel", "termo de serviço"), anote essa informação para auxiliar os próximos agentes.

        Ao final, entregue o texto puro do contrato para o próximo agente.
        """,
        description="Agente que recebe e extrai o texto do contrato."
    )
    return receptor

In [153]:
# Agente 2: Identificador de Jargões
def agente_identificador():
    identificador = Agent(
        name="agente_identificador",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em identificar jargões jurídicos e termos técnicos dentro de um texto de contrato.
        Sua tarefa é analisar o texto do contrato fornecido e listar todos os termos que provavelmente seriam desconhecidos por uma pessoa leiga.

        Considere termos como: latim jurídico (e.g., 'ad hoc'), termos técnicos específicos da área do contrato (e.g., em um contrato de tecnologia, 'API', 'SaaS'), e cláusulas com redação complexa.

        Apresente a lista de jargões identificados de forma clara e concisa para o próximo agente.
        """,
        description="Agente que identifica jargões jurídicos e técnicos."
    )
    return identificador

In [154]:
# Agente 3: Explicador de Termos
def agente_explicador(google_search):
    explicador = Agent(
        name="agente_explicador",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em explicar termos jurídicos e técnicos de forma clara e acessível para pessoas leigas.
        Você receberá uma lista de jargões identificados em um contrato.

        Para cada termo da lista, sua tarefa é fornecer uma explicação concisa e em linguagem simples.
        Se necessário, utilize a ferramenta de busca do Google (google_search) para encontrar definições e explicações adequadas.

        Tente fornecer exemplos práticos ou analogias para facilitar a compreensão.

        Apresente cada termo seguido de sua explicação de forma clara e organizada.
        """,
        description="Agente que explica jargões jurídicos e técnicos para leigos.",
        tools=[google_search]
    )
    return explicador

In [155]:
# Agente 4: Resumidor e Destacador de Pontos Chave
def agente_resumidor():
    resumidor = Agent(
        name="agente_resumidor",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em resumir contratos e destacar os pontos mais importantes para uma pessoa leiga.
        Com base no texto completo do contrato, sua tarefa é gerar um resumo conciso, com linguagem simples e direta.

        Além do resumo, identifique e liste as cláusulas ou pontos que são cruciais para o entendimento do usuário, como:
        - Objeto do contrato
        - Obrigações das partes
        - Prazos e condições
        - Formas de pagamento (se aplicável)
        - Condições de rescisão
        - Penalidades (se houver)

        Apresente o resumo e os pontos chave de forma clara e organizada, utilizando marcadores ou listas para facilitar a leitura.
        """,
        description="Agente que resume contratos e destaca pontos chave."
    )
    return resumidor

In [162]:
print("📜🧐 Iniciando o Sistema Decifra-Contrato (com suporte a arquivos) 📜🧐")

contrato_texto = ""
while True:
    opcao = input("❓ Deseja colar o texto do contrato (digite 'texto') ou carregar um arquivo (digite 'arquivo')? ").lower()
    if opcao in ['texto', 'arquivo']:
        break
    else:
        print("\n⚠️ Opção inválida. Por favor, digite 'texto' ou 'arquivo'.")

if opcao == 'texto':
    contrato_texto = input("Cole o texto do contrato aqui: ")
elif opcao == 'arquivo':
    try:
        uploaded = files.upload()
        if uploaded:
            nome_arquivo = list(uploaded.keys())[0]
            contrato_texto = ler_arquivo(nome_arquivo)
            if isinstance(contrato_texto, str) and contrato_texto.startswith("⚠️"):
                print(f"\n{contrato_texto}")
                contrato_texto = ""
            else:
                print(f"\nArquivo '{nome_arquivo}' lido com sucesso.")
        else:
            print("\n⚠️ Nenhum arquivo foi carregado.")
    except Exception as e:
        print(f"\n⚠️ Ocorreu um erro ao carregar o arquivo: {e}")
        contrato_texto = ""

# Executa o pipeline, independente da origem do texto
if contrato_texto:
    try:
        receptor = agente_receptor()
        identificador = agente_identificador()
        explicador = agente_explicador(google_search)
        resumidor = agente_resumidor()
    except Exception as e:
        print(f"⚠️ Erro ao inicializar os agentes: {e}")
        contrato_texto = ""

if contrato_texto:
    display(Markdown("## ✨ Processando..."))
    texto_contrato_extraido = call_agent(receptor, contrato_texto)
    if texto_contrato_extraido.startswith("⚠️"):
        display(Markdown(f"### ⚠️ Extração: <span style='color:orange'>{texto_contrato_extraido}</span>"))
    else:
        display(Markdown("### 📝 Contrato Extraído"))
        display(to_markdown(texto_contrato_extraido))

        jargoes_identificados = call_agent(identificador, texto_contrato_extraido)
        display(Markdown("### 🔎 Jargões"))
        display(to_markdown(jargoes_identificados))

        explicacoes_termos = call_agent(explicador, jargoes_identificados)
        display(Markdown("### 💡 Explicações"))
        display(to_markdown(explicacoes_termos))

        resumo_contrato = call_agent(resumidor, texto_contrato_extraido)
        display(Markdown("### 📚 Resumo"))
        display(to_markdown(resumo_contrato))

        display(Markdown("## ✅ Análise Concluída!"))

📜🧐 Iniciando o Sistema Decifra-Contrato (com suporte a arquivos) 📜🧐
❓ Deseja colar o texto do contrato (digite 'texto') ou carregar um arquivo (digite 'arquivo')? texto
Cole o texto do contrato aqui: CLÁUSULA	PRIMEIRA	- DO	OBJETO	E	DAS	OBRIGAÇÕES Tendo	em	vista	o	roteiro	e	a	concepção	artística	do	mencionado	filme,	o	CONTRATANTE	 contrata	o	CONTRATADO	para	prestar-lhe	os	serviços	de	técnico	cinematográfico	na	função	de	 _____________________________________. 1.1	A	fim	de	garantir	a	boa	execução	do	trabalho,	o	CONTRATADO	compromete-se,	por	este	 ato,	a	observar	e	cumprir	o	cronograma	de	atividades	da	referida obra.	 1.2	O	CONTRATADO	assume	as	responsabilidades	inerentes	a	sua	função	no	filme	e	se	dispõe	a	 realizá-la	de	acordo	com	o	que	for	requerido	pela	direção	e	produção	da	obra.	 1.3	O	CONTRATADO	cede,	por	prazo	indeterminado	e	de	forma	irretratável	e	irrenunciável	 para	o	CONTRATANTE,	todos	os	direitos	patrimoniais	e	conexos	relativos	ao	seu	trabalho	na	 citada	obra,	que	possam	ser

## ✨ Processando...

### 📝 Contrato Extraído

> CLÁUSULA PRIMEIRA - DO OBJETO E DAS OBRIGAÇÕES Tendo em vista o roteiro e a concepção artística do mencionado filme, o CONTRATANTE contrata o CONTRATADO para prestar-lhe os serviços de técnico cinematográfico na função de _____________________________________. 
> 1.1 A fim de garantir a boa execução do trabalho, o CONTRATADO compromete-se, por este ato, a observar e cumprir o cronograma de atividades da referida obra. 
> 1.2 O CONTRATADO assume as responsabilidades inerentes a sua função no filme e se dispõe a realizá-la de acordo com o que for requerido pela direção e produção da obra. 
> 1.3 O CONTRATADO cede, por prazo indeterminado e de forma irretratável e irrenunciável para o CONTRATANTE, todos os direitos patrimoniais e conexos relativos ao seu trabalho na citada obra, que possam ser auferidos por qualquer utilização do filme, no Brasil e no exterior, para fins comerciais ou não, incluindo a reprodução, gravação, promoção e divulgação da obra, nos meios audiovisuais competentes existentes e a serem criados.
> 


### 🔎 Jargões

...

### 💡 Explicações

> Aqui estão as explicações dos termos jurídicos e técnicos, de forma clara e acessível:
> 
> *   **CONTRATANTE/CONTRATADO:** São os nomes genéricos que usamos para identificar as pessoas ou empresas que assinam um contrato. O "CONTRATANTE" é quem está pagando por um serviço ou produto, e o "CONTRATADO" é quem está oferecendo esse serviço ou produto. Imagine que você está contratando um pedreiro para reformar sua casa: você é o CONTRATANTE, e o pedreiro é o CONTRATADO.
> 
> *   **Direitos patrimoniais e conexos:** Pense nos "direitos patrimoniais" como a capacidade de ganhar dinheiro com a sua criação, como um livro ou uma música. "Direitos conexos" são parecidos, mas protegem quem ajuda a levar a criação ao público, como os atores em um filme ou os músicos em uma gravação.
> 
> *   **Prazo indeterminado:** Significa que não há uma data de término definida para o contrato. Ele continua valendo até que alguma das partes decida encerrá-lo, ou até que algum evento específico aconteça. É como um contrato de aluguel sem data para acabar.
> 
> *   **De forma irretratável e irrenunciável:** Isso quer dizer que, uma vez que você concordou com algo no contrato, não pode voltar atrás (irretratável) nem abrir mão desse acordo (irrenunciável). É como se você vendesse sua casa e prometesse nunca mais reivindicá-la.
> 
> *   **Meios audiovisuais competentes existentes e a serem criados:** Refere-se a todas as formas de comunicação que usam som e imagem, como cinema, televisão, internet, e até tecnologias que ainda nem foram inventadas.
> 
> *   **Inerentes:** Significa que algo é essencial e faz parte da natureza de outra coisa. Por exemplo, a capacidade de voar é inerente aos pássaros.
> 
> *   **Roteiro e a concepção artística:** O "roteiro" é como um mapa detalhado da história que será contada em um filme, série ou peça. A "concepção artística" é a visão geral de como o filme deve ser, incluindo a estética, o estilo e a mensagem que se quer transmitir.
> 
> *   **Cronograma de atividades:** É um plano que mostra todas as tarefas que precisam ser feitas, em que ordem e em quanto tempo. Pense nele como um calendário detalhado de um projeto, mostrando quem faz o quê e quando.
> 


### 📚 Resumo

> ## Resumo do Contrato
> 
> Este contrato estabelece que você (CONTRATADO) foi contratado para trabalhar como técnico cinematográfico na função de [sua função] em um filme. Você deverá seguir o cronograma do filme e realizar suas tarefas conforme solicitado pela direção e produção. Além disso, você cede ao CONTRATANTE todos os direitos sobre seu trabalho no filme, permitindo que ele utilize a obra como quiser, sem limite de tempo.
> 
> ## Pontos Chave do Contrato
> 
> *   **Objeto do Contrato:** Prestação de serviços como técnico cinematográfico na função de [sua função] no filme.
> *   **Obrigações do CONTRATADO:**
>     *   Cumprir o cronograma de atividades do filme.
>     *   Realizar as tarefas de acordo com as instruções da direção e produção.
>     *   Ceder todos os direitos patrimoniais e conexos relativos ao seu trabalho no filme para o CONTRATANTE.
> *   **Prazos e Condições:** O contrato não especifica um prazo determinado, mas a cessão de direitos é por prazo indeterminado.
> *   **Cessão de Direitos:** Você abre mão de todos os direitos sobre seu trabalho no filme, permitindo que o CONTRATANTE utilize a obra como desejar, sem precisar de sua autorização.


## ✅ Análise Concluída!